1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

# ДЗ в процессе выполнения ...

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels
from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier, \
                                        uplift_tree_string, uplift_tree_plot
from catboost import CatBoostClassifier
import warnings

2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [26]:
data = pd.read_csv('../../data/ml_in_business/marketing_uplift/data.csv')
data.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [27]:
data.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
data.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [28]:
data['treatment'].unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

In [29]:
data['treatment'] = data['treatment'].map({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0})
data.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0


3. сделать разбиение набора данных на тренировочную и тестовую выборки

In [37]:
indices_learn, indices_valid = train_test_split(data.index, test_size=0.3, random_state=123)
indices_learn, indices_valid

(Int64Index([53181, 42635,  6296, 41722, 32660, 14730, 40469, 49818, 51773,
             34850,
             ...
             63594,  7763, 15377, 63206, 54370, 61404, 17730, 28030, 15725,
             52734],
            dtype='int64', length=44800),
 Int64Index([19593,  4997, 36682, 29881, 25401, 41888,  4353, 33487, 63951,
             35285,
             ...
             41382, 46836, 36489, 13683, 10904, 39655, 45479, 21007, 39923,
             61435],
            dtype='int64', length=19200))

In [38]:
train_features = data.copy().drop(['target', 'treatment'], axis=1)
train_features

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
0,10,142.44,1,0,Surburban,0,Phone
1,6,329.08,1,1,Rural,1,Web
2,7,180.65,0,1,Surburban,1,Web
3,9,675.83,1,0,Rural,1,Web
4,2,45.34,1,0,Urban,0,Web
...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web
63996,5,38.91,0,1,Urban,1,Phone
63997,6,29.99,1,0,Urban,1,Phone
63998,1,552.94,1,0,Surburban,1,Multichannel


In [44]:
X_train = train_features.loc[indices_learn, :]
y_train = data.loc[indices_learn, 'target']
treat_train = data.loc[indices_learn, 'treatment']

X_test = train_features.loc[indices_valid, :]
y_test = data.loc[indices_valid, 'target']
treat_test =  data.loc[indices_valid, 'treatment']

X_train_full = train_features.loc[data.index, :]
y_train_full = data.loc[:, 'target']
treat_train_full = data.loc[:, 'treatment']

In [52]:
treat_train_full

0        1
1        0
2        1
3        1
4        1
        ..
63995    1
63996    1
63997    1
63998    1
63999    1
Name: treatment, Length: 64000, dtype: int64

4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [53]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': []
}

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion

9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево